In [44]:
import numpy as np
import sklearn

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import ShuffleSplit, GridSearchCV, train_test_split
from sklearn import svm, metrics
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids

In [45]:
# Importamos los datos

X=np.load("./datos_mfcc_pca99_X.npy")
y=np.load("./datos_mfcc_pca99_y.npy")

In [46]:
#Reducimos el conjunto de entrenamiento al 50% usando un método no supervisado. En este caso, un C-means con el que pretendemos
# quedarnos con los centros de los clusters que aparezcan, que serán los puntos más representativos.

c = 6             # número de clases
nc = 120          # muestras por clase
n= 720           # número de muestras totales

X_reduc=[]
y_reduc=np.int16(np.kron(np.arange(c),np.ones(nc//2)))

for i in range(0,n,nc):
    k_means=KMeans(n_clusters=nc//2, random_state=0)
    k_means.fit(X[i:(i+nc)])
    centros=k_means.cluster_centers_
    X_reduc.append(centros)

In [47]:
print(X_reduc[0].shape)
# Reshapeamos las dimensiones de X_reduc para tenerlo todo en una matriz dónde cada fila será una muestra y cada columna una 
# característica. La matriz estará ordenada por clases. 
# Para reshapear, antes habrá que convertir a array.


X_reduc=np.array(X_reduc).reshape((360,464))
print(X_reduc.shape)

(60, 464)
(360, 464)


In [48]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.

Xtr, Xts, ytr, yts = train_test_split(X_reduc, y_reduc, test_size=0.3, shuffle=True, random_state=0)


In [49]:
# Aplicamos directamente un clasificador SVC con los parámetros por defecto y calculamos algunas métricas para ver cómo funciona

clf=svm.SVC(verbose=True)
clf.fit(Xtr,ytr)

score_tr = clf.score(Xtr,ytr)
score_ts = clf.score(Xts,yts)
preds_train=clf.predict(Xtr)
preds_test = clf.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


[LibSVM]
OA train 0.91
Kappa train 0.89
OA test 0.73
Kappa test 0.68


In [50]:
# Matriz de confusión

CM=metrics.confusion_matrix(yts,preds_test)
print(CM)

[[ 8  0  5  0  1  3]
 [ 2 13  0  2  0  3]
 [ 1  0 14  4  0  1]
 [ 2  0  1 19  0  0]
 [ 0  0  2  0 15  1]
 [ 0  0  0  0  1 10]]


Observamos que con los parámetros por defecto, obtenemos unos resultados bastante buenos en entrenamiento y un acierto bastante decente en test. 

Además, vemos que la matriz de confusión está bastante balanceada, por lo que tiene buena pinta este clasificador para nuestros datos.

Veamos ahora qué ocurre si hacemos una búsqueda por cross-validation de los parámetros ideales. Queremos intentar mejorar lo que ya tenemos

In [51]:
# Los parámetros por defecto usados han sido 

clf.get_params() 

# dónde gamma= 'scale' viene definida por la siguiente fórmula: gamma = 1/(464*Xtr.var())

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': True}

In [52]:
1/(464*Xtr.var())

2.001214252382119e-06

In [53]:
# Buscamos los parámetros óptimos con GridSearchCV

gammas = np.logspace(-8, 8, 10)
Cs = np.logspace(-6, 6, 10)  
tuned_parameters = { 'gamma': gammas,'C': Cs} 

clf2 = GridSearchCV(svm.SVC(kernel='rbf'), tuned_parameters, cv=5,n_jobs=-1,verbose=0)
clf2.fit(Xtr,ytr)
clf2=clf2.best_estimator_

score_tr = clf2.score(Xtr,ytr)
score_ts = clf2.score(Xts,yts)
preds_train=clf2.predict(Xtr)
preds_test = clf2.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


OA train 0.98
Kappa train 0.98
OA test 0.75
Kappa test 0.70


In [54]:
clf2

SVC(C=100.0, gamma=5.994842503189409e-07)

In [55]:
clf2.get_params()

{'C': 100.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 5.994842503189409e-07,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [56]:
# Matriz de confusión

CM=metrics.confusion_matrix(yts,preds_test)
print(CM)

[[12  2  1  0  0  2]
 [ 4 13  1  1  0  1]
 [ 1  0 14  4  0  1]
 [ 3  0  1 18  0  0]
 [ 0  0  0  0 17  1]
 [ 0  1  2  0  1  7]]


En general, vemos como al buscar los parámetros óptimos de la función Kernel (='rbf'), los resultados mejoran en todos los aspectos. Obtenemos un mayor acierto tanto en entrenamiento como en test y están balanceados, por lo que en principio no sospecharíamos de sobreajuste. 

Además, al ver la forma de la matriz de confusión volvemos a ver lo mismo, los resultados están muy balanceados.

Ahora vamos a probar a usar el conjunto que hemos obtenido tras eliminar algunas características que no tienen demasiada importancia. Dicho conjunto lo hemos obtenido en el notebook de Árboles. 

Lo cargamos y vemos si mejoran los resultados, o si simplemente no empeoran.

In [57]:
X_reduc2=np.load("./datos_mfcc_pca99_X_reducido_2kmeans.npy")
y_reduc=np.load("./datos_mfcc_pca99_y_reducido_kmeans.npy")

In [61]:
X_reduc2.shape

(360, 5)

In [62]:
y_reduc.shape

(360,)

In [58]:
# Ahora ya podemos dividir nuestro conjunto reducido en entrenamiento y test, y probar algunos métodos Kernel.

Xtr, Xts, ytr, yts = train_test_split(X_reduc2, y_reduc, test_size=0.3, shuffle=True, random_state=0)

In [59]:
# Buscamos los parámetros óptimos con GridSearchCV

gammas = np.logspace(-7, 7, 10)
Cs = np.logspace(-2, 4, 10)  
tuned_parameters = { 'gamma': gammas,'C': Cs} 

clf2 = GridSearchCV(svm.SVC(kernel='rbf'), tuned_parameters, cv=5,n_jobs=-1,verbose=0)
clf2.fit(Xtr,ytr)
clf2=clf2.best_estimator_

score_tr = clf2.score(Xtr,ytr)
score_ts = clf2.score(Xts,yts)
preds_train=clf2.predict(Xtr)
preds_test = clf2.predict(Xts)

print('')
print('OA train %0.2f' % score_tr)
print('Kappa train %0.2f' % metrics.cohen_kappa_score(ytr,preds_train))
print('OA test %0.2f' % score_ts)
print('Kappa test %0.2f' % metrics.cohen_kappa_score(yts,preds_test))


OA train 0.81
Kappa train 0.78
OA test 0.69
Kappa test 0.63


In [60]:
clf2.get_params()

{'C': 4.6415888336127775,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 3.5938136638046257e-06,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

Parece que empeoran los resultados. No sé si merece la pena si lo comparamos con la gran reducción de la dimensionalidad que hemos hecho.

Aún así, siguen siendo mejores resultados que los que obtenemos usando clasificadores del tipo árbol de decisión.